In [1]:
import sys
sys.path.append('..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 6000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [4]:
def binary_search_for_shortest_pulse_time(min_time, max_time, tolerance=1):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns."""
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 * tolerance < max_steps:  # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='Adam', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target:  # if converged, search lower half
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps / 20

----
## H2 (35.3 ns w/ gate based compilation)

In [9]:
circuit = uccsd.get_uccsd_circuit('H2')
U = util.circuitutil.get_unitary(circuit)

N = 2
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [10]:
shortest_time = binary_search_for_shortest_pulse_time(1.5, 4.0, tolerance=0.3)



trying total_time: 2.75 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00027_Mar26.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Saved model at: /tmp/grape4339.cpkt
Error = :9.93e-01; Runtime: 0.1s; Iterations = 0, grads =   4.511e-04, unitary_metric = 1.00000
Error = :7.18e-01; Runtime: 0.4s; Iterations = 20, grads =   2.939e-02, unitary_metric = 1.00000
Error = :4.35e-01; Runtime: 0.5s; Iterations = 40, grads =   1.191e-02, unitary_metric = 1.00000
Error = :2.84e-01; Runtime: 0.7s; Iterations = 60, grads =   2.110e-03, unitary_metric = 1.00000
Error = :2.41e-01; Runtime: 0.8s; Iterations = 80, grads =   8.490e-04, unitary_metric = 1.00000
Error = :2.05e-01; Runtime: 0.9s; Iterati

Error = :4.13e-02; Runtime: 10.8s; Iterations = 1580, grads =   4.875e-07, unitary_metric = 1.00000
Error = :4.12e-02; Runtime: 11.0s; Iterations = 1600, grads =   4.439e-07, unitary_metric = 1.00000
Error = :4.12e-02; Runtime: 11.2s; Iterations = 1620, grads =   4.046e-07, unitary_metric = 1.00000
Error = :4.12e-02; Runtime: 11.3s; Iterations = 1640, grads =   3.697e-07, unitary_metric = 1.00000
Error = :4.12e-02; Runtime: 11.4s; Iterations = 1660, grads =   3.394e-07, unitary_metric = 1.00000
Error = :4.11e-02; Runtime: 11.5s; Iterations = 1680, grads =   3.132e-07, unitary_metric = 1.00000
Error = :4.11e-02; Runtime: 11.6s; Iterations = 1700, grads =   2.908e-07, unitary_metric = 1.00000
Error = :4.11e-02; Runtime: 11.8s; Iterations = 1720, grads =   2.720e-07, unitary_metric = 1.00000
Error = :4.11e-02; Runtime: 11.9s; Iterations = 1740, grads =   2.561e-07, unitary_metric = 1.00000
Error = :4.11e-02; Runtime: 12.1s; Iterations = 1760, grads =   2.428e-07, unitary_metric = 1.00000


Error = :4.04e-02; Runtime: 21.9s; Iterations = 3220, grads =   9.762e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.0s; Iterations = 3240, grads =   9.602e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.1s; Iterations = 3260, grads =   9.446e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.2s; Iterations = 3280, grads =   9.290e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.3s; Iterations = 3300, grads =   9.135e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.5s; Iterations = 3320, grads =   8.981e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.6s; Iterations = 3340, grads =   8.827e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.7s; Iterations = 3360, grads =   8.675e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 22.9s; Iterations = 3380, grads =   8.519e-08, unitary_metric = 1.00000
Error = :4.03e-02; Runtime: 23.0s; Iterations = 3400, grads =   8.362e-08, unitary_metric = 1.00000


Error = :3.79e-02; Runtime: 32.8s; Iterations = 4860, grads =   1.957e-06, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 32.9s; Iterations = 4880, grads =   2.570e-07, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.1s; Iterations = 4900, grads =   3.028e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.3s; Iterations = 4920, grads =   2.701e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.4s; Iterations = 4940, grads =   2.452e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.5s; Iterations = 4960, grads =   2.292e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.6s; Iterations = 4980, grads =   2.208e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.7s; Iterations = 5000, grads =   2.179e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 33.9s; Iterations = 5020, grads =   2.196e-08, unitary_metric = 1.00000
Error = :3.79e-02; Runtime: 34.0s; Iterations = 5040, grads =   2.086e-08, unitary_metric = 1.00000


Error = :3.64e-03; Runtime: 3.3s; Iterations = 400, grads =   3.227e-06, unitary_metric = 1.00000
Error = :3.31e-03; Runtime: 3.6s; Iterations = 420, grads =   2.630e-06, unitary_metric = 1.00000
Error = :3.04e-03; Runtime: 3.7s; Iterations = 440, grads =   2.175e-06, unitary_metric = 1.00000
Error = :2.81e-03; Runtime: 3.8s; Iterations = 460, grads =   1.822e-06, unitary_metric = 1.00000
Error = :2.61e-03; Runtime: 4.0s; Iterations = 480, grads =   1.544e-06, unitary_metric = 1.00000
Error = :2.44e-03; Runtime: 4.1s; Iterations = 500, grads =   1.321e-06, unitary_metric = 1.00000
Error = :2.30e-03; Runtime: 4.3s; Iterations = 520, grads =   1.140e-06, unitary_metric = 1.00000
Error = :2.17e-03; Runtime: 4.5s; Iterations = 540, grads =   9.922e-07, unitary_metric = 1.00000
Error = :2.05e-03; Runtime: 4.6s; Iterations = 560, grads =   8.695e-07, unitary_metric = 1.00000
Error = :1.95e-03; Runtime: 4.7s; Iterations = 580, grads =   7.670e-07, unitary_metric = 1.00000
Error = :1.86e-03; R

Error = :1.85e-02; Runtime: 6.4s; Iterations = 840, grads =   1.230e-06, unitary_metric = 1.00000
Error = :1.83e-02; Runtime: 6.5s; Iterations = 860, grads =   1.146e-06, unitary_metric = 1.00000
Error = :1.82e-02; Runtime: 6.6s; Iterations = 880, grads =   1.069e-06, unitary_metric = 1.00000
Error = :1.81e-02; Runtime: 6.7s; Iterations = 900, grads =   9.984e-07, unitary_metric = 1.00000
Error = :1.80e-02; Runtime: 6.9s; Iterations = 920, grads =   9.342e-07, unitary_metric = 1.00000
Error = :1.79e-02; Runtime: 7.1s; Iterations = 940, grads =   8.767e-07, unitary_metric = 1.00000
Error = :1.78e-02; Runtime: 7.2s; Iterations = 960, grads =   8.256e-07, unitary_metric = 1.00000
Error = :1.77e-02; Runtime: 7.3s; Iterations = 980, grads =   7.806e-07, unitary_metric = 1.00000
Error = :1.76e-02; Runtime: 7.4s; Iterations = 1000, grads =   7.410e-07, unitary_metric = 1.00000
Error = :1.76e-02; Runtime: 7.6s; Iterations = 1020, grads =   7.064e-07, unitary_metric = 1.00000
Error = :1.75e-02;

Error = :1.48e-02; Runtime: 18.0s; Iterations = 2500, grads =   1.043e-07, unitary_metric = 1.00000
Error = :1.48e-02; Runtime: 18.3s; Iterations = 2520, grads =   1.033e-07, unitary_metric = 1.00000
Error = :1.48e-02; Runtime: 18.4s; Iterations = 2540, grads =   1.024e-07, unitary_metric = 1.00000
Error = :1.48e-02; Runtime: 18.5s; Iterations = 2560, grads =   1.014e-07, unitary_metric = 1.00000
Error = :1.48e-02; Runtime: 18.6s; Iterations = 2580, grads =   1.004e-07, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 18.7s; Iterations = 2600, grads =   9.940e-08, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 19.0s; Iterations = 2620, grads =   9.840e-08, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 19.1s; Iterations = 2640, grads =   9.734e-08, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 19.2s; Iterations = 2660, grads =   9.631e-08, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 19.3s; Iterations = 2680, grads =   9.527e-08, unitary_metric = 1.00000


Error = :1.44e-02; Runtime: 29.6s; Iterations = 4140, grads =   5.940e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 29.8s; Iterations = 4160, grads =   5.819e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 29.9s; Iterations = 4180, grads =   5.711e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.0s; Iterations = 4200, grads =   5.595e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.2s; Iterations = 4220, grads =   5.490e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.3s; Iterations = 4240, grads =   5.391e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.5s; Iterations = 4260, grads =   5.302e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.6s; Iterations = 4280, grads =   5.211e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.7s; Iterations = 4300, grads =   5.131e-08, unitary_metric = 1.00000
Error = :1.44e-02; Runtime: 30.9s; Iterations = 4320, grads =   5.052e-08, unitary_metric = 1.00000


Error = :1.42e-02; Runtime: 41.3s; Iterations = 5780, grads =   1.747e-06, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 41.4s; Iterations = 5800, grads =   2.581e-07, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 41.6s; Iterations = 5820, grads =   1.982e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 41.7s; Iterations = 5840, grads =   2.159e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 41.9s; Iterations = 5860, grads =   1.816e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 42.0s; Iterations = 5880, grads =   1.771e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 42.1s; Iterations = 5900, grads =   1.721e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 42.3s; Iterations = 5920, grads =   1.783e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 42.4s; Iterations = 5940, grads =   4.197e-08, unitary_metric = 1.00000
Error = :1.42e-02; Runtime: 42.6s; Iterations = 5960, grads =   2.829e-06, unitary_metric = 1.00000


In [11]:
print(shortest_time)

3.05


----
## LiH (871.7 ns w/ gate based compilation)

In [13]:
circuit = uccsd.get_uccsd_circuit('LiH')
U = util.circuitutil.get_unitary(circuit)

N = 4
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = np.zeros((d ** N, d ** N))
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [18]:
shortest_time = binary_search_for_shortest_pulse_time(18.0, 22.0, tolerance=0.3)



trying total_time: 20.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00035_Mar26.h5
Using 7 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Saved model at: /tmp/grape52591.cpkt
Error = :9.79e-01; Runtime: 0.5s; Iterations = 0, grads =   1.481e-04, unitary_metric = 1.00001
Error = :7.08e-01; Runtime: 2.5s; Iterations = 20, grads =   5.672e-02, unitary_metric = 1.00000
Error = :5.62e-01; Runtime: 3.3s; Iterations = 40, grads =   1.809e-02, unitary_metric = 1.00000
Error = :5.44e-01; Runtime: 4.2s; Iterations = 60, grads =   1.342e-03, unitary_metric = 1.00000
Error = :5.36e-01; Runtime: 5.0s; Iterations = 80, grads =   1.913e-04, unitary_metric = 1.00000
Error = :5.25e-01; Runtime: 5.9s; Iter

Error = :6.57e-03; Runtime: 76.7s; Iterations = 1560, grads =   6.525e-07, unitary_metric = 0.99992
Error = :6.39e-03; Runtime: 77.6s; Iterations = 1580, grads =   6.367e-07, unitary_metric = 0.99992
Error = :6.22e-03; Runtime: 78.5s; Iterations = 1600, grads =   7.778e-07, unitary_metric = 0.99992
Error = :6.06e-03; Runtime: 80.3s; Iterations = 1620, grads =   5.791e-05, unitary_metric = 0.99993
Error = :7.48e-03; Runtime: 81.2s; Iterations = 1640, grads =   1.902e-02, unitary_metric = 0.99992
Error = :5.81e-03; Runtime: 82.0s; Iterations = 1660, grads =   6.101e-04, unitary_metric = 0.99992
Error = :5.63e-03; Runtime: 82.9s; Iterations = 1680, grads =   8.783e-05, unitary_metric = 0.99993
Error = :5.49e-03; Runtime: 83.7s; Iterations = 1700, grads =   2.800e-05, unitary_metric = 0.99992
Error = :5.37e-03; Runtime: 85.1s; Iterations = 1720, grads =   5.178e-06, unitary_metric = 0.99992
Error = :5.24e-03; Runtime: 86.0s; Iterations = 1740, grads =   8.891e-07, unitary_metric = 0.99993


Error = :1.70e-03; Runtime: 156.4s; Iterations = 3200, grads =   3.664e-06, unitary_metric = 0.99992
Error = :1.72e-03; Runtime: 157.8s; Iterations = 3220, grads =   5.037e-04, unitary_metric = 0.99993
Error = :1.72e-03; Runtime: 158.6s; Iterations = 3240, grads =   6.446e-04, unitary_metric = 0.99992
Error = :1.65e-03; Runtime: 159.5s; Iterations = 3260, grads =   1.056e-04, unitary_metric = 0.99992
Error = :1.62e-03; Runtime: 160.3s; Iterations = 3280, grads =   3.230e-07, unitary_metric = 0.99993
Error = :1.60e-03; Runtime: 161.2s; Iterations = 3300, grads =   3.106e-06, unitary_metric = 0.99993
Error = :1.58e-03; Runtime: 162.5s; Iterations = 3320, grads =   6.751e-07, unitary_metric = 0.99992
Error = :1.56e-03; Runtime: 163.4s; Iterations = 3340, grads =   9.358e-08, unitary_metric = 0.99993
Error = :1.54e-03; Runtime: 164.2s; Iterations = 3360, grads =   7.661e-08, unitary_metric = 0.99992
Error = :1.52e-03; Runtime: 165.1s; Iterations = 3380, grads =   7.125e-08, unitary_metric 

Error = :7.37e-02; Runtime: 26.5s; Iterations = 580, grads =   1.054e-04, unitary_metric = 0.99994
Error = :6.97e-02; Runtime: 27.3s; Iterations = 600, grads =   2.647e-03, unitary_metric = 0.99994
Error = :6.60e-02; Runtime: 28.6s; Iterations = 620, grads =   2.450e-03, unitary_metric = 0.99994
Error = :6.26e-02; Runtime: 29.3s; Iterations = 640, grads =   2.388e-04, unitary_metric = 0.99994
Error = :5.96e-02; Runtime: 30.1s; Iterations = 660, grads =   1.060e-04, unitary_metric = 0.99994
Error = :5.69e-02; Runtime: 30.9s; Iterations = 680, grads =   2.015e-03, unitary_metric = 0.99994
Error = :5.47e-02; Runtime: 31.6s; Iterations = 700, grads =   6.337e-03, unitary_metric = 0.99993
Error = :5.17e-02; Runtime: 32.9s; Iterations = 720, grads =   6.661e-04, unitary_metric = 0.99994
Error = :4.93e-02; Runtime: 33.7s; Iterations = 740, grads =   1.214e-04, unitary_metric = 0.99993
Error = :4.71e-02; Runtime: 34.4s; Iterations = 760, grads =   1.818e-05, unitary_metric = 0.99993
Error = :4

Error = :5.99e-03; Runtime: 97.9s; Iterations = 2240, grads =   1.305e-05, unitary_metric = 0.99993
Error = :5.87e-03; Runtime: 98.7s; Iterations = 2260, grads =   4.357e-06, unitary_metric = 0.99993
Error = :5.76e-03; Runtime: 99.4s; Iterations = 2280, grads =   2.861e-05, unitary_metric = 0.99993
Error = :5.88e-03; Runtime: 100.2s; Iterations = 2300, grads =   2.450e-03, unitary_metric = 0.99993
Error = :5.63e-03; Runtime: 101.5s; Iterations = 2320, grads =   8.612e-04, unitary_metric = 0.99993
Error = :5.45e-03; Runtime: 102.2s; Iterations = 2340, grads =   3.373e-05, unitary_metric = 0.99993
Error = :5.35e-03; Runtime: 103.0s; Iterations = 2360, grads =   1.415e-05, unitary_metric = 0.99993
Error = :5.26e-03; Runtime: 103.7s; Iterations = 2380, grads =   2.033e-06, unitary_metric = 0.99993
Error = :5.17e-03; Runtime: 104.5s; Iterations = 2400, grads =   5.182e-06, unitary_metric = 0.99993
Error = :5.11e-03; Runtime: 105.8s; Iterations = 2420, grads =   2.627e-04, unitary_metric = 0

...

Error = :3.24e-03; Runtime: 121.5s; Iterations = 2680, grads =   6.517e-04, unitary_metric = 0.99992
Error = :3.18e-03; Runtime: 122.3s; Iterations = 2700, grads =   5.990e-04, unitary_metric = 0.99992
Error = :3.08e-03; Runtime: 123.6s; Iterations = 2720, grads =   2.105e-05, unitary_metric = 0.99992
Error = :3.03e-03; Runtime: 124.4s; Iterations = 2740, grads =   4.379e-05, unitary_metric = 0.99992
Error = :3.02e-03; Runtime: 125.2s; Iterations = 2760, grads =   5.780e-04, unitary_metric = 0.99992
Error = :2.96e-03; Runtime: 126.0s; Iterations = 2780, grads =   4.596e-04, unitary_metric = 0.99992
Error = :2.89e-03; Runtime: 126.8s; Iterations = 2800, grads =   4.573e-05, unitary_metric = 0.99992
Error = :2.84e-03; Runtime: 128.2s; Iterations = 2820, grads =   1.761e-05, unitary_metric = 0.99992
Error = :2.79e-03; Runtime: 128.9s; Iterations = 2840, grads =   8.017e-06, unitary_metric = 0.99992
Error = :2.80e-03; Runtime: 129.7s; Iterations = 2860, grads =   4.865e-04, unitary_metric 

Error = :1.42e-03; Runtime: 196.4s; Iterations = 4320, grads =   2.752e-03, unitary_metric = 0.99993
Error = :1.20e-03; Runtime: 197.2s; Iterations = 4340, grads =   2.756e-04, unitary_metric = 0.99992
Error = :1.17e-03; Runtime: 197.9s; Iterations = 4360, grads =   3.247e-06, unitary_metric = 0.99993
Error = :1.16e-03; Runtime: 198.7s; Iterations = 4380, grads =   1.262e-06, unitary_metric = 0.99993
Error = :1.15e-03; Runtime: 199.5s; Iterations = 4400, grads =   1.242e-05, unitary_metric = 0.99993
Error = :1.18e-03; Runtime: 200.8s; Iterations = 4420, grads =   3.893e-04, unitary_metric = 0.99993
Error = :1.13e-03; Runtime: 201.6s; Iterations = 4440, grads =   1.502e-05, unitary_metric = 0.99993
Error = :1.12e-03; Runtime: 202.4s; Iterations = 4460, grads =   2.942e-06, unitary_metric = 0.99992
Error = :1.11e-03; Runtime: 203.2s; Iterations = 4480, grads =   6.649e-07, unitary_metric = 0.99993
Error = :1.10e-03; Runtime: 204.0s; Iterations = 4500, grads =   1.961e-07, unitary_metric 

Error = :1.85e-02; Runtime: 51.2s; Iterations = 1120, grads =   4.003e-03, unitary_metric = 0.99994
Error = :1.79e-02; Runtime: 52.0s; Iterations = 1140, grads =   3.313e-03, unitary_metric = 0.99993
Error = :1.71e-02; Runtime: 52.8s; Iterations = 1160, grads =   1.185e-04, unitary_metric = 0.99993
Error = :1.66e-02; Runtime: 53.5s; Iterations = 1180, grads =   1.150e-05, unitary_metric = 0.99993
Error = :1.61e-02; Runtime: 54.3s; Iterations = 1200, grads =   4.566e-05, unitary_metric = 0.99993
Error = :1.57e-02; Runtime: 55.6s; Iterations = 1220, grads =   2.394e-03, unitary_metric = 0.99993
Error = :1.53e-02; Runtime: 56.4s; Iterations = 1240, grads =   2.934e-03, unitary_metric = 0.99993
Error = :1.47e-02; Runtime: 57.1s; Iterations = 1260, grads =   6.446e-04, unitary_metric = 0.99993
Error = :1.42e-02; Runtime: 57.9s; Iterations = 1280, grads =   3.557e-05, unitary_metric = 0.99993
Error = :1.38e-02; Runtime: 58.7s; Iterations = 1300, grads =   2.061e-05, unitary_metric = 0.99993


Error = :3.17e-03; Runtime: 123.0s; Iterations = 2760, grads =   1.075e-03, unitary_metric = 0.99993
Error = :3.06e-03; Runtime: 123.8s; Iterations = 2780, grads =   3.810e-06, unitary_metric = 0.99993
Error = :3.03e-03; Runtime: 124.6s; Iterations = 2800, grads =   3.124e-05, unitary_metric = 0.99993
Error = :2.99e-03; Runtime: 125.8s; Iterations = 2820, grads =   1.028e-06, unitary_metric = 0.99993
Error = :2.95e-03; Runtime: 126.6s; Iterations = 2840, grads =   3.325e-07, unitary_metric = 0.99993
Error = :2.92e-03; Runtime: 127.4s; Iterations = 2860, grads =   3.944e-07, unitary_metric = 0.99993
Error = :2.89e-03; Runtime: 128.2s; Iterations = 2880, grads =   9.649e-07, unitary_metric = 0.99993
Error = :2.86e-03; Runtime: 129.0s; Iterations = 2900, grads =   4.128e-05, unitary_metric = 0.99993
Error = :3.27e-03; Runtime: 130.3s; Iterations = 2920, grads =   6.289e-03, unitary_metric = 0.99993
Error = :2.80e-03; Runtime: 131.0s; Iterations = 2940, grads =   1.624e-04, unitary_metric 

Error = :1.39e-03; Runtime: 195.0s; Iterations = 4400, grads =   1.010e-06, unitary_metric = 0.99993
Error = :1.39e-03; Runtime: 196.3s; Iterations = 4420, grads =   1.843e-04, unitary_metric = 0.99993
Error = :1.38e-03; Runtime: 197.0s; Iterations = 4440, grads =   1.514e-04, unitary_metric = 0.99993
Error = :1.36e-03; Runtime: 197.8s; Iterations = 4460, grads =   4.627e-05, unitary_metric = 0.99993
Error = :1.35e-03; Runtime: 198.6s; Iterations = 4480, grads =   9.135e-06, unitary_metric = 0.99993
Error = :1.33e-03; Runtime: 199.3s; Iterations = 4500, grads =   1.095e-07, unitary_metric = 0.99993
Error = :1.33e-03; Runtime: 200.6s; Iterations = 4520, grads =   5.525e-07, unitary_metric = 0.99993
Error = :1.32e-03; Runtime: 201.4s; Iterations = 4540, grads =   3.320e-07, unitary_metric = 0.99993
Error = :1.31e-03; Runtime: 202.2s; Iterations = 4560, grads =   2.494e-05, unitary_metric = 0.99993
Error = :1.37e-03; Runtime: 202.9s; Iterations = 4580, grads =   8.866e-04, unitary_metric 

In [19]:
print(shortest_time)

19.25


----
## BeH2 (5308.3 ns w/ gate based compilation)

This ansatz has 6 qubits. We can only handle up to 4 qubits at a time with QOC, so the solution is to divide the BeH2 circuit into blocks of up to 4 qubits at time.
Under the layout,
0 1 2
5 4 3
, the blocks cycle through {0123, 2345, 1234}. Each block was run in parallel on RCC. The script below just reads the results and sums up the times for each block.

In [89]:
import os
times = []
for filename in os.listdir('BeH2/'):
    if filename.endswith('.out'):
        lines = open('BeH2/%s' % filename).readlines()
        if len(lines) > 5 and lines[-1].startswith('^^^SHORTEST TIME was '):
            result = lines[-1]
            time = float(result[len('^^^SHORTEST TIME was '):])
            if time > 50:
                print(time, filename)
            else:
                times.append(time)
            
print(sum(times))
print(len(times))

59.75 arrayJob_58942771_93.out
59.75 arrayJob_58942771_53.out
59.75 arrayJob_58942771_77.out
59.75 arrayJob_58942771_244.out
59.75 arrayJob_58942771_75.out
59.75 arrayJob_58942771_91.out
59.75 arrayJob_58942771_51.out
59.75 arrayJob_58942771_267.out
59.75 arrayJob_58942771_252.out
59.75 arrayJob_58942771_67.out
59.75 arrayJob_58942771_265.out
59.75 arrayJob_58942771_235.out
2762.2499999999973
338


In [90]:
# all of the 59.75 times were due to a bug. They are all 2-qubit subcircuits
# but missing connectivity in the GRAPE that I ran. I am just treating upper bounding
# their times as 2 * SWAP = 14.8
times += [14.8] * 12

In [91]:
print(sum(times))

2939.8499999999995


----
## NaH (5490.4 ns w/ gate based compilation)

----
## H2O (33842.2 ns w/ gate based compilation)